In [3]:
!pip install spotipy

  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 4.0.1
    Uninstalling async-timeout-4.0.1:
      Successfully uninstalled async-timeout-4.0.1


In [1]:
import requests
import base64

#Spotify Client ID ans Secret ID
CLIENT_ID = 'XXXXX'
CLIENT_SECRET = 'XXXXXX'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [26]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [20]:
playlist_id = '4NRpl56XvyWIULGamewgld'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

In [19]:
# Display the DataFrame
music_df.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Ram Ram,"Kamal Haasan, Shruti Haasan",Heyram (Original Motion Picture Soundtrack),5qhBG5YYYn9qeI6mDjeGii,5iJvZOhgRxbXCd1NTZr9v4,37,2000-05-01,282006,False,https://open.spotify.com/track/5iJvZOhgRxbXCd1...,...,0.697,9,-10.131,0,0.0774,0.0483,0.009580,0.1500,0.487,94.125
1,Marugo Marugo,"Kamal Haasan, K. S. Chithra, Kovai Sarala",Sathi Leelavathi,2IO7xMMIU7aHYAjMveF55b,0vp4FqDh9Mk2NHmubHgCse,0,1995-01-15,369912,False,https://open.spotify.com/track/0vp4FqDh9Mk2NHm...,...,0.469,2,-10.429,1,0.1540,0.6560,0.000094,0.0895,0.523,126.024
2,Anbe Sivam,"Kamal Haasan, Karthik",Anbe Sivam (Original Motion Picture Soundtrack),6poMR702Ml9KMgHVVr9HwG,3ieFvgn8ALw1gf3Q5YjtPV,0,2003-01-14,261093,False,https://open.spotify.com/track/3ieFvgn8ALw1gf3...,...,0.355,7,-13.405,0,0.0308,0.2630,0.000130,0.0949,0.505,149.805
3,Kadhal Piriyamal,Kamal Haasan,Panchathanthiram (Original Motion Picture Soun...,4Rfi7APRCZRUIjJLlZ7Pjw,1RzUVImXNzYx87PpmMEdhQ,16,2002-12-01,358027,False,https://open.spotify.com/track/1RzUVImXNzYx87P...,...,0.629,10,-13.487,0,0.0304,0.0258,0.000138,0.0668,0.625,98.992
4,Vandhaen Vandhaen,"Kamal Haasan, Sujatha, Nithyasree Mahadevan",Panchathanthiram (Original Motion Picture Soun...,4Rfi7APRCZRUIjJLlZ7Pjw,1aaLtG3D4YvhWwc9rvHF1R,22,2002-12-01,340573,False,https://open.spotify.com/track/1aaLtG3D4YvhWwc...,...,0.672,1,-13.852,1,0.0525,0.1700,0.000225,0.1370,0.722,108.992


In [24]:
music_df.columns

Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Explicit',
       'External URLs', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo'],
      dtype='object')

In [9]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [11]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [12]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [14]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [15]:
input_song_name = "Vandhaen Vandhaen"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Vandhaen Vandhaen':
                        Track Name  \
2  Pathala Pathala (From "Vikram")   
4              Raja Kaiya Vachchaa   
0                  Andha Kandamani   
1                      Allah Jaane   
3              Kalakapovathu Yaaru   

                                             Artists  \
2                  Anirudh Ravichander, Kamal Haasan   
4                                       Kamal Haasan   
0  Ilaiyaraaja, Kamal Haasan, Karthik, Tippu, Sur...   
1                        Shruti Haasan, Kamal Haasan   
3                              Kamal Haasan, Sathyan   

                                          Album Name Release Date  Popularity  
2                    Pathala Pathala (From "Vikram")   2022-05-11        70.0  
4  Apoorva Sagodharargal (Original Motion Picture...   1989-04-14        47.0  
0                                          Virumandi   2003-01-10        37.0  
1  Unnai Pol Oruvan (Original Motion Picture Soun...   2009-09-0

C:\Users\krish\AppData\Local\Temp\ipykernel_10164\209674984.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({


In [16]:
input_song_name = "Marugo Marugo"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Marugo Marugo':
            Track Name                       Artists  \
1            Unna Vida  Kamal Haasan, Shreya Ghoshal   
2  Raja Kaiya Vachchaa                  Kamal Haasan   
3      Thanmaanam Ulla            Kamal Haasan Hasan   
0      Nam Ganga Nathi            Kamal Haasan Hasan   
4      Kadavaul Paadhi         Kamal Haasan, NANDINI   

                                          Album Name Release Date  Popularity  
1                                          Virumandi   2003-01-10        63.0  
2  Apoorva Sagodharargal (Original Motion Picture...   1989-04-14        47.0  
3                                           Mahanadi   1994-01-14        46.0  
0                                           Mahanadi   1994-01-14        37.0  
4   Aalavanthan (Original Motion Picture Soundtrack)         2001         0.0  


C:\Users\krish\AppData\Local\Temp\ipykernel_10164\209674984.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
